## Demographics and Projects Upkeep

**To-Do**
* Change file name, messed up.

In [1]:
# importing necessary modules
import pandas as pd
import numpy as np

In [2]:
# reading and printing the head to see what's inside
df = pd.read_excel("data/All demographics and programs.xlsx")

display(df)

,Auto Id,First Name,Last Name,Gender,Race,Ethnicity Hispanic/Latino,Outcome,Veteran,Ex-Offender,Justice Involved,Single Parent,Program: Program Name
0,202107-1206,name,name,Male,Black or African American,NaN,NaN,No,NaN,NaN,NaN,Reimage 21-22
1,202107-1206,name,name,Male,Black or African American,NaN,NaN,No,NaN,NaN,NaN,Reimage 21-22
2,202107-1206,name,name,Male,Black or African American,NaN,NaN,No,NaN,NaN,NaN,Reimage 21-22
3,202108-5167,name,name,Male,Asian,NaN,Successfully Completed,No,NaN,No,NaN,Tech Louisville 21-22
4,202108-5171,name,name,Male,Black or African American,NaN,NaN,NaN,NaN,NaN,NaN,Tech Louisville 21-22
...,...,...,...,...,...,...,...,...,...,...,...,...
32225,202502-20671,name,name,Female,White,NaN,NaN,NaN,NaN,NaN,NaN,Connecting Young Adults 24-25
32226,202410-17602,name,name,Female,White,NaN,NaN,NaN,NaN,NaN,NaN,Connecting Young Adults 24-25
32227,202506-23809,name,name,Female,White,NaN,NaN,NaN,NaN,NaN,NaN,Connecting Young Adults 24-25
32228,202410-17749,name,name,Female,White,NaN,NaN,NaN,NaN,NaN,NaN,Connecting Young Adults 24-25


In [3]:
# upon first look-through I saw that some rows were either blank or sparse, checking how many entries are in each column
df.count()

Auto Id                      32230
First Name                   32230
Last Name                    32230
Gender                       31621
Race                         31121
Ethnicity Hispanic/Latino        0
Outcome                      15852
Veteran                      19955
Ex-Offender                      1
Justice Involved             10235
Single Parent                    1
Program: Program Name        32230
dtype: int64

#### Clean Up
* Duplicate items in 'Auto Id', drop them
* Three columns are either sparsely populated or completely empty (error?)
    - Drop 'Ethnicity Hispanic/Latino' as there are no entries.
    - Drop 'Ex-Offender' and 'Single Parent' as there is only 1 such instance of both among 13,000 entries making them beyond a statistical outlier - <1%.

*What does 'Justice Involved' mean?*  Ask, for now let's remove.

In [4]:
def df_cleaning(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop_duplicates(ignore_index=True) # drop_duplicates checks all elements in a row at default, so we drop the extras and keep the index
    
    min_pct = 0.01
    df = df.dropna(axis=1, thresh=int(min_pct * len(df))) # set the threshold for viable data which is >1%

    columnsToRemove = [
        'First Name',
        'Last Name',
        ] # possible keep depending on further conversations?
    df = df.drop(columnsToRemove, axis=1) # remove columns: 'First Name', 'Last Name', and 'Justice Involved'
    
    df = df.rename(columns={ # renaming to tidy things up
        "Auto Id": "Auto ID",
        "Program: Program Name": "Program Name"
        })

    fillElement = "Not Provided"    
    df = df.replace(np.nan, fillElement) # changing 'NaN' to fillElement

    return(df)

###### *Important Note: there's only 1 instance of a Single Parent and 1 of an Ex-Offender.  Those, while not statistically relevent, are noteworthy.  Additionally, there were no hispanics that showed up in the original data file, so that column has been removed as well.*

#### Showcase the New DF

In [5]:
df_cleaned = df_cleaning(df)
display(df_cleaned)

,Auto ID,Gender,Race,Outcome,Veteran,Justice Involved,Program Name
0,202107-1206,Male,Black or African American,Not Provided,No,Not Provided,Reimage 21-22
1,202108-5167,Male,Asian,Successfully Completed,No,No,Tech Louisville 21-22
2,202108-5171,Male,Black or African American,Not Provided,Not Provided,Not Provided,Tech Louisville 21-22
3,202108-5172,Female,White,Not Provided,Not Provided,Not Provided,Reimage 21-22
4,202107-1208,Female,Black or African American,Not Provided,Not Provided,Not Provided,Reimage 21-22
...,...,...,...,...,...,...,...
13670,202506-23973,Female,White,Not Provided,Not Provided,Not Provided,Connecting Young Adults 24-25
13671,202506-23877,Female,Black or African American,Not Provided,Not Provided,Not Provided,Connecting Young Adults 24-25
13672,202506-23967,Male,White,Not Provided,Not Provided,Not Provided,Connecting Young Adults 24-25
13673,202506-23976,Female,White,Not Provided,Not Provided,Not Provided,Connecting Young Adults 24-25


In [6]:
df_cleaned.to_csv('data/cleaned/cleaned demographics_and_programs.csv', index=False) # saving the cleaned file